In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from sklearn import model_selection
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
%matplotlib inline

Using TensorFlow backend.


In [2]:
Corpus = pd.read_csv(r"D:\\NLP project\\dataset\\final\\cleandata2.csv",encoding='latin-1')
Corpus['title'].dropna(inplace=True)

In [3]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(Corpus['title'],Corpus['label'],test_size=0.3,shuffle=True)

In [4]:
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y.astype(str))
Test_Y = Encoder.fit_transform(Test_Y.astype(str))


In [5]:
training_labels_final = np.array(Train_Y)
testing_labels_final = np.array(Test_Y)

In [6]:
max_words = 500000
max_len = 200
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(Train_X)
sequences = tok.texts_to_sequences(Train_X)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [7]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,32,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [8]:
APPLY_ATTENTION_BEFORE_LSTM = False
SINGLE_ATTENTION_VECTOR = False
def attention_3d_block(inputs):
    # inputs.shape = (batch_size, time_steps, input_dim)
    input_dim = int(inputs.shape[2])
    a = Permute((2, 1))(inputs)
    a = Reshape((input_dim, TIME_STEPS))(a) # this line is not useful. It's just to know which dimension is what.
    a = Dense(TIME_STEPS, activation='softmax')(a)
    if SINGLE_ATTENTION_VECTOR:
        a = Lambda(lambda x: K.mean(x, axis=1), name='dim_reduction')(a)
        a = RepeatVector(input_dim)(a)
    a_probs = Permute((2, 1), name='attention_vec')(a)
    # output_attention_mul = merge([inputs, a_probs], name='attention_mul', mode='mul')
    output_attention_mul = multiply([inputs, a_probs])
    return output_attention_mul

In [9]:
def model_attention_applied_after_lstm():
    #inputs = Input(shape=(TIME_STEPS, INPUT_DIM,))
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    
    lstm_units = 64
    lstm_out = LSTM(lstm_units, return_sequences=True)(layer)
    attention_mul = attention_3d_block(lstm_out)
    attention_mul = Flatten()(attention_mul)
    output = Dense(1, activation='sigmoid')(attention_mul)
    model = Model(input=[inputs], output=output)
    return model

In [10]:
from keras.layers import merge
from keras.layers import multiply
from keras.layers.core import *
from keras.layers.recurrent import LSTM
from keras.models import *

from keras.utils.vis_utils import plot_model

In [11]:
INPUT_DIM = 50
TIME_STEPS = max_len

In [12]:
sequences_matrix.shape,Train_Y.shape

((1693088, 200), (1693088,))

In [13]:
m = model_attention_applied_after_lstm()

D:\anaconda\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`
  # This is added back by InteractiveShellApp.init_path()


In [14]:
m.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inputs (InputLayer)             (None, 200)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 200, 50)      25000000    inputs[0][0]                     
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, 200, 64)      29440       embedding_1[0][0]                
__________________________________________________________________________________________________
permute_1 (Permute)             (None, 64, 200)      0           lstm_1[0][0]                     
____________________________________________________________________________________________

In [15]:
m.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
m.fit(sequences_matrix,Train_Y,batch_size=128,epochs=1,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 1354470 samples, validate on 338618 samples
Epoch 1/1





1354470/1354470 [==============================] - 6673s 5ms/step - loss: 0.0380 - acc: 0.9856 - val_loss: 0.0210 - val_acc: 0.9932


In [17]:
test_sequences = tok.texts_to_sequences(Test_X)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [18]:
accr = m.evaluate(test_sequences_matrix,Test_Y)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

725610/725610 [==============================] - 552s 761us/step
Test set
  Loss: 0.022
  Accuracy: 0.993


In [37]:
sentence = ["nice weather"]
sequences = tok.texts_to_sequences(sentence)
padded = sequence.pad_sequences(sequences, maxlen=max_len)
x=m.predict(padded)
print(x)
if x<0.3:
    print("inapprpriate context")
else:
    print("appropriate context")

[[0.6067322]]
appropriate context


In [38]:
import pickle
with open('D:\\NLP project\\tokenizerkagglemodelattention.pickle', 'wb') as handle:
    pickle.dump(tok, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [39]:
m.save('D:\\NLP project\\models\\LSTMkagglemodelattention')